In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

f = open('Finetwork KOI - Equipo - Superliga.html', 'r', encoding="UTF8")
html = f.read()
f.close()

# First of all get the data from our webside

In [3]:
soup = BeautifulSoup(html,'html')

In [4]:
display(soup.find_all('div',{'class' : 'players-container-inner'}))
all_players = soup.find_all('a',{'class' : 'player-card'})

[<div class="players-container-inner"> <a class="player-card" href="https://superliga.lvp.global/jugador/sinmivak"> <!-- PLAYER --><div class="squad-player-container"><div class="upper-player-card"><div class="photo"> <img alt="" class="player-image" src="./Finetwork KOI - Equipo - Superliga_files/6467481d75e8c816090991.x2.png"/></div><div class="upper-player-info-container"><div class="flag-image-container"> <img alt="Player flag nationality" class="flag-image" src="./Finetwork KOI - Equipo - Superliga_files/PL.png"/></div><h3><span class="player-nickname">Sinmivak</span></h3><h4><span class="player-name">Jakub Rucki</span></h4><h4><span class="player-position">TOP</span></h4></div></div><div class="bottom-player-card"><div class="stats-container"> <img alt="Best player champion image." src="./Finetwork KOI - Equipo - Superliga_files/Sion.png" title="Sion"/> <img alt="Best player champion image." src="./Finetwork KOI - Equipo - Superliga_files/Renekton.png" title="Renekton"/> <img alt

In [5]:
stats_all = []
for player in all_players[:5]:
    stats_all.append(player.find('span', {'class':'stats-player'}))
    

In [23]:
stats_all[0].text.replace('-','').replace('KDA','').strip().split()

i = 0
for player in stats_all:
    stats_all[i] = player.text.replace('-','').replace('KDA','').strip().split()
    i += 1

In [24]:
stats_all

df = pd.DataFrame(stats_all)

df

,0,1
0,44/34/104,"4,4"
1,38/46/170,"4,5"
2,81/48/122,"4,2"
3,98/56/109,"3,7"
4,22/51/204,"4,4"
